# Prepare Data
Here we prepare data as 1 min level kline for BTC from 2019.1.1 to 2020.5.2 in bitfinex exchange

In [1]:
#!python3 -m pip install -e .. -U
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta
from sklearn.metrics import roc_curve, auc

In [2]:
TZ = 'Asia/Shanghai'

In [3]:
data_path = './data/data'
data_platform_list = ['BITFINEX']
data_symbol_list = ['BTC']

data_df_list = []

In [4]:
for platform in data_platform_list:
    for symbol in data_symbol_list:
        pkl_file_path = data_path+'/'+symbol+'_USD_'+platform+'_latest.pkl'
        pandas_df = pd.read_pickle(pkl_file_path)
        #data_df_list.append(pkl_file.add_prefix(platform+'_'+symbol+':'))
        data_df_list.append(pandas_df)
data = pd.concat(data_df_list, axis=1)

# Feature Enginnering

Tactical indicators etc.

In [5]:
data['timestamp'] = data.index.tolist()

In [6]:
data

,open,high,low,close,volume,timestamp
2019-01-01 16:00:00+08:00,3850.000000,3850.000000,3849.600000,3849.600000,0.443293,2019-01-01 16:00:00+08:00
2019-01-01 16:01:00+08:00,3849.500000,3853.000000,3849.400000,3853.000000,9.085920,2019-01-01 16:01:00+08:00
2019-01-01 16:02:00+08:00,3853.000000,3857.000000,3852.960000,3853.200000,8.213360,2019-01-01 16:02:00+08:00
2019-01-01 16:03:00+08:00,3853.000000,3853.100000,3851.200000,3852.200000,6.385190,2019-01-01 16:03:00+08:00
2019-01-01 16:04:00+08:00,3852.200000,3852.300000,3852.200000,3852.300000,0.504622,2019-01-01 16:04:00+08:00
...,...,...,...,...,...,...
2020-11-07 15:55:00+08:00,15428.349581,15437.000000,15428.000000,15436.374894,1.040375,2020-11-07 15:55:00+08:00
2020-11-07 15:56:00+08:00,15434.000000,15447.000000,15434.000000,15443.000000,5.073000,2020-11-07 15:56:00+08:00
2020-11-07 15:57:00+08:00,15441.000000,15446.000000,15441.000000,15446.000000,1.608000,2020-11-07 15:57:00+08:00
2020-11-07 15:58:00+08:00,15443.786243,15443.786243,15430.460723,15430.460723,0.692501,2020-11-07 15:58:00+08:00


In [7]:
# Prepare features
import talib

# Moving averages
data['ma5m'] = talib.MA(data['close'], timeperiod = 5) / data['close'] 
data['ma10m'] = talib.MA(data['close'], timeperiod = 10) / data['close'] 
data['ma1h'] = talib.MA(data['close'], timeperiod = 60) / data['close'] 
data['ma4h'] = talib.MA(data['close'], timeperiod = 240) / data['close'] 
data['ma12h'] = talib.MA(data['close'], timeperiod = 720) / data['close'] 
data['ma1d'] = talib.MA(data['close'], timeperiod = 1440) / data['close']
data['ma5d'] = talib.MA(data['close'], timeperiod = 7200) / data['close'] 
data['ma10d'] = talib.MA(data['close'], timeperiod = 14400) / data['close'] 
data['ma30d'] = talib.MA(data['close'], timeperiod = 43200) / data['close'] 


# Standard deviation
data['std5m'] = talib.STDDEV(data['close'], timeperiod=5)/ data['close'] 
data['std10m'] = talib.STDDEV(data['close'], timeperiod = 10) / data['close'] 
data['std1h'] = talib.STDDEV(data['close'], timeperiod = 60) / data['close'] 
data['std4h'] = talib.STDDEV(data['close'], timeperiod = 240) / data['close'] 
data['std12h'] = talib.STDDEV(data['close'], timeperiod = 720) / data['close'] 
data['std1d'] = talib.STDDEV(data['close'], timeperiod = 1440) / data['close']
data['std5d'] = talib.STDDEV(data['close'], timeperiod = 7200) / data['close'] 
data['std10d'] = talib.STDDEV(data['close'], timeperiod = 14400) / data['close'] 
data['std30d'] = talib.STDDEV(data['close'], timeperiod = 43200) / data['close'] 

# Closeness to hundred / thousand
data['dis100'] = (data['close'] % 100) / 100 
data['dis1000'] = (data['close'] % 1000) / 1000 



In [8]:
data['up_5%_in_1d_label'] = data['close'] > data['close'].shift(-1440) * 1.05

In [9]:
data=data.dropna()

# Simple Strategy Example

## Prepare data feed

# Machine Learning Strategy Example

refernces: https://towardsdatascience.com/model-design-and-selection-with-scikit-learn-18a29041d02a

## Dataset Preparation

In [10]:
train_start_time = pd.Timestamp('2019-01-01', tz=TZ)
train_end_time = pd.Timestamp('2019-12-31', tz=TZ)
val_start_time = pd.Timestamp('2020-01-01', tz=TZ)
val_end_time = pd.Timestamp('2020-06-01', tz=TZ)
test_start_time = pd.Timestamp('2020-06-01', tz=TZ)
test_end_time = pd.Timestamp('2020-11-01', tz=TZ)

train_data = data.loc[train_start_time:train_end_time]
val_data = data.loc[train_start_time:train_end_time]
test_data = data.loc[test_start_time:test_end_time]

In [11]:
feature_set = ['ma5m','ma10m','ma1h','ma4h','ma12h','ma1d','ma5d','ma10d','ma30d', \
               'std5m','std10m','std1h','std4h','std12h','std1d','std5d','std10d','std30d',\
               'dis100', 'dis1000']
label = ['up_5%_in_1d_label']

X_train = train_data[feature_set]
y_train = train_data[label]
X_val = train_data[feature_set]
y_val = train_data[label]
X_test = test_data[feature_set]
y_test = test_data[label]

In [12]:
y_train.sum() / y_train.count()

up_5%_in_1d_label    0.062166
dtype: float64

## Define model and hyperparameter

In [13]:
# For reading, visualizing, and preprocessing data
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.feature_selection import RFE, RFECV
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.ensemble import GradientBoostingClassifier

### Initialize Sample Reweighting

In [14]:
from sample_reweight import SampleReweight

In [15]:
LIMITED_SAMPLE = True
EXAMPLES = 40000 if LIMITED_SAMPLE else X_train.shape[0]

# number of submodels
K = 10

# initialize sample weights and parameters
w = np.ones(X_train.shape[0])[:EXAMPLES]

ALPHA_1 = 1
ALPHA_2 = 1
NUM_BINS = 10
GAMMA = 0.9

# initialize sample reweighting
SR = SampleReweight(X_train[:EXAMPLES], y_train[:EXAMPLES].to_numpy(dtype=int), a1=ALPHA_1, a2=ALPHA_2, b=NUM_BINS, gamma=GAMMA)

### Define model

In [16]:
clfs = [GradientBoostingClassifier(random_state=0) for i in range(K)]

### Train Sub-model Loops

In [17]:
try:
    from tqdm import tqdm
    submodels = tqdm(clfs)
except ImportError:
    submodels = clfs
    
for model in submodels:
    model.fit(X_train[:EXAMPLES], y_train[:EXAMPLES], sample_weight=w)
    w = SR.reweight(model)

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]C:\Users\Yuan\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


[[0.5627]
 [0.5669]
 [0.567 ]
 ...
 [0.048 ]
 [0.046 ]
 [0.0354]]
[[0.5627]
 [0.5669]
 [0.567 ]
 ...
 [0.048 ]
 [0.046 ]
 [0.0354]]
[[1.000e-04]
 [6.664e-01]
 [6.665e-01]
 ...
 [3.337e-01]
 [3.330e-01]
 [1.000e+00]]
(10000,)
(10000,)
[1. 1. 1. ... 1. 1. 1.]


C:\Users\Yuan\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3334: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Yuan\anaconda3\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
 10%|████████▎                                                                          | 1/10 [00:04<00:42,  4.67s/it]C:\Users\Yuan\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


[10. 10. 10. ... 10. 10. 10.]
[[0.5627]
 [0.5669]
 [0.567 ]
 ...
 [0.048 ]
 [0.046 ]
 [0.0354]]
[[0.5627 0.5627]
 [0.5669 0.5669]
 [0.567  0.567 ]
 ...
 [0.048  0.048 ]
 [0.046  0.046 ]
 [0.0354 0.0354]]
[[1.000e-04]
 [6.664e-01]
 [6.665e-01]
 ...
 [3.337e-01]
 [3.330e-01]
 [1.000e+00]]
(10000,)
(10000,)
[1. 1. 1. ... 1. 1. 1.]


C:\Users\Yuan\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3334: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Yuan\anaconda3\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
 20%|████████████████▌                                                                  | 2/10 [00:09<00:37,  4.66s/it]C:\Users\Yuan\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


[10. 10. 10. ... 10. 10. 10.]
[[0.5627]
 [0.5669]
 [0.567 ]
 ...
 [0.048 ]
 [0.046 ]
 [0.0354]]
[[0.5627 0.5627 0.5627]
 [0.5669 0.5669 0.5669]
 [0.567  0.567  0.567 ]
 ...
 [0.048  0.048  0.048 ]
 [0.046  0.046  0.046 ]
 [0.0354 0.0354 0.0354]]
[[1.000e-04]
 [6.664e-01]
 [6.665e-01]
 ...
 [3.337e-01]
 [3.330e-01]
 [1.000e+00]]
(10000,)
(10000,)
[1. 1. 1. ... 1. 1. 1.]
[10. 10. 10. ... 10. 10. 10.]

C:\Users\Yuan\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3334: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Yuan\anaconda3\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
 30%|████████████████████████▉                                                          | 3/10 [00:13<00:32,  4.64s/it]C:\Users\Yuan\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


 30%|████████████████████████▉                                                          | 3/10 [00:17<00:41,  5.92s/it]


KeyboardInterrupt: 

In [ ]:
y_score = np.mean(np.assarray([clf.decision_function(X_test) for clf in clfs]))

## Evaluate Model

In [ ]:
def plot_roc(labels, scores):
    lw = 2
    fpr, tpr, _ = roc_curve(labels, scores)
    roc_auc = auc(fpr, tpr)
    plt.figure(figsize=(5,5))
    plt.plot(fpr, tpr, color='darkorange', lw=lw, label='(AUC = %0.3f)' % (roc_auc))
    #plt.plot([eer], [1-eer], marker='o', markersize=5, color="navy")
    #plt.plot([0, 1], [1, 0], color='navy', lw=1, linestyle=':')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic Curve')
    plt.legend(loc="lower right")

In [ ]:
plot_roc(y_test, y_score)